In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import numba
# import ray
# %env MODIN_ENGINE=ray
# import modin.pandas as mpd

In [2]:
# ray.init()

2023-11-26 11:50:43,511	INFO worker.py:1673 -- Started a local Ray instance.


Python version:,3.11.5
Ray version:,2.8.0


(raylet) [2023-11-26 11:50:53,417 E 12247 12260] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-11-26_11-50-41_363763_12097 is over 95% full, available space: 113524736; capacity: 3602239488. Object creation will fail if spilling is required.
(raylet) [2023-11-26 11:51:03,424 E 12247 12260] (raylet) file_system_monitor.cc:111: /tmp/ray/session_2023-11-26_11-50-41_363763_12097 is over 95% full, available space: 113508352; capacity: 3602239488. Object creation will fail if spilling is required.


In [2]:
train_df = pd.read_csv('train.csv', engine="pyarrow")

In [3]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24236806 entries, 0 to 24236805
Data columns (total 10 columns):
 #   Column     Dtype  
---  ------     -----  
 0   timestamp  int64  
 1   Asset_ID   int64  
 2   Count      float64
 3   Open       float64
 4   High       float64
 5   Low        float64
 6   Close      float64
 7   Volume     float64
 8   VWAP       float64
 9   Target     float64
dtypes: float64(8), int64(2)
memory usage: 1.8 GB


In [4]:
train_df.head(5)

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [5]:
asset_details = pd.read_csv('asset_details.csv')

In [7]:
asset_details.head(), asset_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14 entries, 0 to 13
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Asset_ID    14 non-null     int64  
 1   Weight      14 non-null     float64
 2   Asset_Name  14 non-null     object 
dtypes: float64(1), int64(1), object(1)
memory usage: 468.0+ bytes


(   Asset_ID    Weight        Asset_Name
 0         2  2.397895      Bitcoin Cash
 1         0  4.304065      Binance Coin
 2         1  6.779922           Bitcoin
 3         5  1.386294            EOS.IO
 4         7  2.079442  Ethereum Classic,
 None)

In [8]:
btc = train_df[train_df['Asset_ID'] == 1].set_index("timestamp")

In [9]:
btc_mini = btc.iloc[-200:]

In [12]:
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=btc.index, open=btc['Open'],
                high=btc['High'], low=btc['Low'], close=btc['Close'])])
fig.show()

In [5]:
test_df = pd.read_csv('example_test.csv')

In [6]:
asset_details = pd.read_csv('asset_details.csv')

In [7]:
asset_details

,Asset_ID,Weight,Asset_Name
0,2,2.397895,Bitcoin Cash
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
3,5,1.386294,EOS.IO
4,7,2.079442,Ethereum Classic
5,6,5.894403,Ethereum
6,9,2.397895,Litecoin
7,11,1.609438,Monero
8,13,1.791759,TRON
9,12,2.079442,Stellar


In [6]:
train_df

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.580000,2399.500000,2357.140000,2374.590000,1.923301e+01,2373.116392,-0.004218
1,1514764860,0,5.0,8.530000,8.530000,8.530000,8.530000,7.838000e+01,8.530000,-0.014399
2,1514764860,1,229.0,13835.194000,14013.800000,13666.110000,13850.176000,3.155006e+01,13827.062093,-0.014643
3,1514764860,5,32.0,7.659600,7.659600,7.656700,7.657600,6.626713e+03,7.657713,-0.013922
4,1514764860,7,5.0,25.920000,25.920000,25.874000,25.877000,1.210873e+02,25.891363,-0.008264
...,...,...,...,...,...,...,...,...,...,...
24236801,1632182400,9,775.0,157.181571,157.250000,156.700000,156.943857,4.663725e+03,156.994319,NaN
24236802,1632182400,10,34.0,2437.065067,2438.000000,2430.226900,2432.907467,3.975460e+00,2434.818747,NaN
24236803,1632182400,13,380.0,0.091390,0.091527,0.091260,0.091349,2.193732e+06,0.091388,NaN
24236804,1632182400,12,177.0,0.282168,0.282438,0.281842,0.282051,1.828508e+05,0.282134,NaN


In [8]:
test_df

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,group_num,row_id
0,1623542400,3,1201.0,1.478556,1.486030,1.478000,1.483681,6.547996e+05,1.481439,0,0
1,1623542400,2,1020.0,580.306667,583.890000,579.910000,582.276667,1.227988e+03,581.697038,0,1
2,1623542400,0,626.0,343.789500,345.108000,343.640000,344.598000,1.718833e+03,344.441729,0,2
3,1623542400,1,2888.0,35554.289632,35652.464650,35502.670000,35602.004286,1.638115e+02,35583.469303,0,3
4,1623542400,4,433.0,0.312167,0.312600,0.311920,0.312208,5.855774e+05,0.312154,0,4
5,1623542400,5,359.0,4.832550,4.845900,4.822900,4.837583,4.714355e+04,4.836607,0,5
6,1623542400,7,541.0,55.223080,55.494000,55.182000,55.344680,6.625202e+03,55.298816,0,6
7,1623542400,6,2186.0,2371.194286,2379.200000,2369.670000,2374.380714,1.214129e+03,2374.335307,0,7
8,1623542400,8,35.0,1.003150,1.019800,0.987300,1.003300,7.061928e+03,1.002936,0,8
9,1623542400,9,560.0,161.933429,162.480000,161.730000,162.214714,1.485009e+03,162.231310,0,9


In [ ]:
plt.plot()